Dokumentation Game Entwicklung

Um eine solide Basis für das Spiel zu bekommen, haben wir als Erstes die Datei mit den Vorgaben hochgeladen und den Prompt "Erstelle anhand dieser Vorgaben ein Spiel. Das Spiel soll wie Mario aufgebaut sein, es soll 4 Level haben. In den Leveln sollen sich Gegner befinden, die Schwierigkeit soll sich außerdem steigern."

In [ ]:
public class Main {
    public static void main(String[] args) {
        javax.swing.SwingUtilities.invokeLater(() -> {
            Game.createAndShowGUI();
        });
    }
}

In [ ]:
import javax.swing.*;
import java.awt.*;


public class Game {
    private static JFrame frame;
    public static void createAndShowGUI() {
        frame = new JFrame("Platformer - MarioStyle Project");
        frame.setDefaultCloseOperation(JFrame.EXIT_ON_CLOSE);
        frame.setResizable(false);


        GamePanel panel = new GamePanel();
        frame.setContentPane(panel);
        frame.pack();
        frame.setLocationRelativeTo(null);
        frame.setVisible(true);


        panel.startGame();
    }
}

In [ ]:
import javax.swing.*;
import java.awt.*;
import java.awt.event.*;
import java.util.ArrayList;


// Main panel: handles loop, rendering, input, state
public class GamePanel extends JPanel implements ActionListener, KeyListener {
    public static final int WIDTH = 800;
    public static final int HEIGHT = 600;


    private Timer timer;
    private boolean left, right, jump, restart, startPressed;


    private Player player;
    private ArrayList<Enemy> enemies;
    private Level level;
    private int currentLevelIndex = 0;
    private int score = 0;
    private int lives = 3;


    private enum State { START, RUNNING, GAME_OVER, LEVEL_COMPLETE }
    private State state = State.START;


    public GamePanel() {
        setPreferredSize(new Dimension(WIDTH, HEIGHT));
        setBackground(Color.CYAN);
        setFocusable(true);
        addKeyListener(this);


        timer = new Timer(16, this); // ~60 FPS


        loadLevel(currentLevelIndex);
    }


    public void startGame() {
        timer.start();
    }


    private void loadLevel(int index) {
        level = Level.createSampleLevel(index);
        player = new Player(50, 400, level);
        enemies = new ArrayList<>();
// create enemies based on level
        for (int i = 0; i < level.getEnemyPositions().length; i++) {
        int[] p = level.getEnemyPositions()[i];
        enemies.add(new Enemy(p[0], p[1], level));
    }
        state = State.START;
    }

    @Override
    public void actionPerformed(ActionEvent e) {
// game loop update
        if (state == State.RUNNING) {
            handleInput();
            player.update();
            for (Enemy en : enemies) en.update();


// collisions enemy-player
            for (Enemy en : new ArrayList<>(enemies)) {
                if (player.getBounds().intersects(en.getBounds())) {
                    if (player.isFalling() && player.getY() < en.getY()) {
// stomp
                        enemies.remove(en);
                        score += 100;
                        player.bounceAfterStomp();
                    } else {
// take damage
                        lives--;
                        if (lives <= 0) {
                            state = State.GAME_OVER;
                        } else {
// respawn player at start of level
                            player.respawn();
                        }
                    }
                }
            }


// check level end: reach end tile
            if (level.isEndReached(player)) {
                state = State.LEVEL_COMPLETE;
            }
        }


        repaint();
    }


    private void handleInput() {
        if (left) player.moveLeft();
        if (right) player.moveRight();
        if (!left && !right) player.stopHorizontal();
        if (jump) { player.jump(); jump = false; }
    }

    @Override
    protected void paintComponent(Graphics g) {
        super.paintComponent(g);
        Graphics2D g2 = (Graphics2D) g;


// draw level (tiles)
        level.draw(g2);


// draw player
        player.draw(g2);


// draw enemies
        for (Enemy en : enemies) en.draw(g2);


// UI - score and lives
        g2.setColor(Color.BLACK);
        g2.setFont(new Font("SansSerif", Font.BOLD, 18));
        g2.drawString("Score: " + score, 10, 20);
        g2.drawString("Lives: " + lives, 10, 40);
        g2.drawString("Level: " + (currentLevelIndex + 1), 700, 20);


// state overlays
        if (state == State.START) {
            drawCenteredString(g2, "PRESS ENTER TO START", getWidth(), getHeight());
            drawCenteredString(g2, "Controls: Arrow keys to move, SPACE to jump", getWidth(), getHeight()-30);
        } else if (state == State.GAME_OVER) {
            drawCenteredString(g2, "GAME OVER - Press R to restart", getWidth(), getHeight());
        } else if (state == State.LEVEL_COMPLETE) {
            drawCenteredString(g2, "LEVEL COMPLETE - Press ENTER to continue", getWidth(), getHeight());
        }
    }


    private void drawCenteredString(Graphics2D g, String text, int w, int h) {
        FontMetrics fm = g.getFontMetrics();
        int x = (w - fm.stringWidth(text)) / 2;
        int y = (h - fm.getHeight()) / 2 + fm.getAscent();
        g.setColor(Color.WHITE);
        g.fillRect(x-10, y-fm.getAscent()-5, fm.stringWidth(text)+20, fm.getHeight()+10);
        g.setColor(Color.BLACK);
        g.drawString(text, x, y);
    }


// KeyListener
    @Override
    public void keyPressed(KeyEvent e) {
        int k = e.getKeyCode();
        if (k == KeyEvent.VK_LEFT || k == KeyEvent.VK_A) left = true;
        if (k == KeyEvent.VK_RIGHT || k == KeyEvent.VK_D) right = true;
        if (k == KeyEvent.VK_SPACE || k == KeyEvent.VK_UP || k == KeyEvent.VK_W) jump = true;
        if (k == KeyEvent.VK_R) { // restart
            if (state == State.GAME_OVER) {
                score = 0; lives = 3; currentLevelIndex = 0; loadLevel(currentLevelIndex); state = State.START;
            }
        }
        if (k == KeyEvent.VK_ENTER) {
            if (state == State.START) { state = State.RUNNING; }
            else if (state == State.LEVEL_COMPLETE) {
                currentLevelIndex++;
                if (currentLevelIndex >= Level.NUM_LEVELS) {
// all levels done -> game over / victory
                    state = State.GAME_OVER;
                } else {
                    loadLevel(currentLevelIndex);
                    state = State.START;
                }
            }
        }
    }


    @Override
    public void keyReleased(KeyEvent e) {
        int k = e.getKeyCode();
        if (k == KeyEvent.VK_LEFT || k == KeyEvent.VK_A) left = false;
        if (k == KeyEvent.VK_RIGHT || k == KeyEvent.VK_D) right = false;
    }


    @Override
    public void keyTyped(KeyEvent e) {}
}

In [ ]:
import java.awt.*;


public class Player {
    private double x, y;
    private double vx, vy;
    private int w = 32, h = 48;
    private Level level;
    private boolean onGround = false;


    private final double GRAVITY = 0.8;
    private final double MOVE_SPEED = 3.0;
    private final double JUMP_SPEED = -14.0;


    private final double MAX_FALL = 20.0;


    private final double startX, startY;


    public Player(double startX, double startY, Level level) {
        this.x = startX; this.y = startY; this.startX = startX; this.startY = startY; this.level = level;
    }


    public void update() {
// apply gravity
        vy += GRAVITY;
        if (vy > MAX_FALL) vy = MAX_FALL;


// horizontal movement
        x += vx;
// horizontal collisions
        Rectangle bounds = getBounds();
        for (Tile t : level.getSolidTiles()) {
            if (t.getRect().intersects(bounds)) {
                if (vx > 0) x = t.getX() - w;
                else if (vx < 0) x = t.getX() + t.getW();
                vx = 0;
                bounds = getBounds();
            }
        }


// vertical movement
        y += vy;
        bounds = getBounds();
        onGround = false;
        for (Tile t : level.getSolidTiles()) {
            if (t.getRect().intersects(bounds)) {
                if (vy > 0) { // falling
                    y = t.getY() - h;
                    onGround = true;
                } else if (vy < 0) {
                    y = t.getY() + t.getW();
                }
                vy = 0;
                bounds = getBounds();
            }
        }


// keep inside world bounds
        if (x < 0) x = 0;
        if (x + w > level.getWidth()) x = level.getWidth() - w;
        if (y > level.getHeight()) { // fell off
            respawn();
        }
    }

    public void moveLeft() { vx = -MOVE_SPEED; }
    public void moveRight() { vx = MOVE_SPEED; }
    public void stopHorizontal() { vx = 0; }
    public void jump() {
        if (onGround) { vy = JUMP_SPEED; onGround = false; }
    }


    public void bounceAfterStomp() { vy = JUMP_SPEED/2; }


    public void respawn() {
        x = startX; y = startY; vx = 0; vy = 0;
    }


    public boolean isFalling() { return vy > 0; }


    public Rectangle getBounds() { return new Rectangle((int)x, (int)y, w, h); }


    public int getY() { return (int)y; }


    public void draw(Graphics2D g) {
        g.setColor(Color.RED);
        g.fillRect((int)x, (int)y, w, h);
// head eye
        g.setColor(Color.WHITE);
        g.fillRect((int)x + 8, (int)y + 8, 6, 6);
        g.setColor(Color.BLACK);
        g.fillRect((int)x + 10, (int)y + 10, 2, 2);
    }
}

In [ ]:
import java.awt.*;
public class Enemy {
    private double x, y;
    private double vx = 1.5;
    private int w = 32, h = 32;
    private Level level;
    public Enemy(double x, double y, Level level) {
        this.x = x; this.y = y; this.level = level;
    }
    public void update() {
        x += vx;
// turn around on collisions or edges
        Rectangle bounds = getBounds();
        boolean collided = false;
        for (Tile t : level.getSolidTiles()) {
            if (t.getRect().intersects(bounds)) {
                collided = true; break;
            }
        }
        if (collided) {
            vx = -vx;
            x += vx * 2; // step back
        }
// gravity simple
        y += 4;
        bounds = getBounds();
        for (Tile t : level.getSolidTiles()) {
            if (t.getRect().intersects(bounds)) {
                y = t.getY() - h;
            }
        }
    }
    public Rectangle getBounds() { return new Rectangle((int)x, (int)y, w,
        h); }
    public int getY() { return (int)y; }
    public void draw(Graphics2D g) {
        g.setColor(Color.ORANGE);
        g.fillOval((int)x, (int)y, w, h);
    }
}

In [ ]:
import java.awt.*;
import java.util.ArrayList;
public class Level {
    public static final int NUM_LEVELS = 4;
    private int width, height;
    private ArrayList<Tile> tiles = new ArrayList<>();
    private int[][] layout;
    private int tileSize = 40;
    private int[][] enemyPositions;
    private Rectangle endRect;
    private Level(int width, int height) {
        this.width = width; this.height = height;
    }
    public static Level createSampleLevel(int idx) {
        Level L = new Level(2000, 600);
// simple layout arrays: 0 empty, 1 solid block, 2 end flag
// We'll produce different patterns per level
        int rows = 15;
        int cols = L.width / L.tileSize;
        int[][] mat = new int[rows][cols];
        // base ground
        for (int c = 0; c < cols; c++) mat[rows-2][c] = 1;
// add platforms differently per level
        if (idx == 0) {
// easy: few platforms
            mat[11][5] = 1; mat[9][8] = 1; mat[11][12] = 1; mat[10][18] = 1;
            L.enemyPositions = new int[][]{{300, 440},{700,440}};
            L.endRect = new Rectangle((cols-3)*L.tileSize,
            (rows-3)*L.tileSize, L.tileSize, L.tileSize);
        } else if (idx == 1) {
// more platforms
            for (int i=6;i<cols;i+=10) mat[10][i]=1;
            for (int i=3;i<cols;i+=14) mat[8][i]=1;
            L.enemyPositions = new int[][]{{500,440},{900,440},{1200,440}};
            L.endRect = new Rectangle((cols-4)*L.tileSize,
            (rows-3)*L.tileSize, L.tileSize, L.tileSize);
        } else if (idx == 2) {
// moving gaps
            for (int i=0;i<cols;i++) if (i%6!=0) mat[rows-2][i]=1; // holes
            for (int i=5;i<cols;i+=12) mat[9][i]=1; mat[7][15]=1; mat[7]
            [25]=1;
            L.enemyPositions = new int[][]{{600,440},{950,240},{1400,440},
                {1600,240}};
            L.endRect = new Rectangle((cols-5)*L.tileSize,
            (rows-5)*L.tileSize, L.tileSize, L.tileSize);
        } else {
// dangerous final level
            for (int c=0;c<cols;c++) mat[rows-2][c]=1;
            for (int i=8;i<cols;i+=6) mat[11][i]=1;
            for (int i=4;i<cols;i+=9) mat[9][i]=1;
            L.enemyPositions = new int[][]{{400,440},{700,440},{1100,440},
                {1500,440},{1700,440}};
            L.endRect = new Rectangle((cols-6)*L.tileSize,
            (rows-6)*L.tileSize, L.tileSize, L.tileSize);
        }
// convert mat to tiles
        for (int r=0;r<rows;r++) for (int c=0;c<cols;c++) if (mat[r][c]==1) {
            int x = c*L.tileSize; int y = r*L.tileSize;
            L.tiles.add(new Tile(x,y,L.tileSize,L.tileSize));
        }
        return L;
    }
    public ArrayList<Tile> getSolidTiles() { return tiles; }
    public int getWidth() { return width; }
    public int getHeight() { return height; }
    public int[][] getEnemyPositions() { return enemyPositions; }
    public void draw(Graphics2D g) {
// camera follows player? Simple: draw at fixed offset (0) so full
        world visible horizontally
// draw background
        g.setColor(new Color(135, 206, 235));
        g.fillRect(0,0,width,height);
// draw tiles
        g.setColor(new Color(100, 60, 20));
        for (Tile t : tiles) {
            g.fillRect(t.getX(), t.getY(), t.getW(), t.getH());
        }
// draw end flag
        g.setColor(Color.MAGENTA);
        g.fillRect(endRect.x, endRect.y, endRect.width, endRect.height);
// optionally draw simple background elements
        g.setColor(Color.YELLOW);
        g.fillOval(700,40,80,80);
    }
    public boolean isEndReached(Player p) {
        return endRect.intersects(p.getBounds());
    }
}


In [ ]:
import java.awt.*;
public class Tile {
    private int x,y,w,h;
    public Tile(int x,int y,int w,int h) { this.x=x; this.y=y; this.w=w;
        this.h=h; }
    public Rectangle getRect() {
        return new Rectangle(x,y,w,h);
    }

    public int getX(){return x;}
    public int getY(){return y;}
    public int getW(){return w;}
    public int getH(){return h;}
}
